#Install dependencies

You can use a GPU accelerated notebook for faster generation

In [1]:
!pip install transformers==4.19.2
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 108.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.5 MB/s eta 0:00:00
Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 19.34 MiB/s, done.
Resolving deltas: 100% (339/339), done.


# Imports

In [2]:
#textwrap enables formating of long text
import textwrap

from transformers import pipeline, GPT2TokenizerFast
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from arabert.preprocess import ArabertPreprocessor

#you can choose any aragpt2 model since they all have the same preprocessing
arabert_processor = ArabertPreprocessor(model_name="aragpt2-base")

Setting up device

In [3]:
!nvidia-smi

Sat Jun 17 19:38:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
device = 0 if torch.cuda.is_available() else -1
print(device)

0


# Setting up pipelines

For base and medium model, you can just provide the model name to the pipeline function

In [5]:
model_name = "aubmindlab/aragpt2-base"

aragpt2_pipeline = pipeline("text-generation",model=model_name,device=device)

Downloading:   0%|          | 0.00/843 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

For large and mega models, you need to provide the custom model and tokenizer to the class

In [ ]:
model_name = "aubmindlab/aragpt2-large" #the mega model needs a High-Ram colab

grover_gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

aragpt2_pipeline = pipeline("text-generation",model=grover_gpt2_model, tokenizer=tokenizer,device=device)

Downloading:   0%|          | 0.00/923 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.98G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

# Start Generating

In [20]:
text = """
ما معنى القدر
"""

In [21]:
text_prep = arabert_processor.preprocess(text)
print(text_prep)

ما معنى القدر


For more info about the parameter check this blog https://huggingface.co/blog/how-to-generate and the docummentation for the generate function https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate

In [22]:
%%time
gen_text = aragpt2_pipeline(text_prep,
            pad_token_id=0, # 0 for AraGPT2
            num_beams=5,
            max_length=200,
            top_p=0.95,
            repetition_penalty = 3.0,
            no_repeat_ngram_size = 3)[0]['generated_text']

CPU times: user 3.5 s, sys: 7.71 ms, total: 3.51 s
Wall time: 3.49 s


In [24]:
gen_text

'ما معنى القدر ؟ قال : " إن الله لا يقبض العلم انتزاعا ينتزعه من العباد ، ولكن يقبض العلم بقبض العلماء ، حتى إذا لم يبق عالما اتخذ الناس رؤوسا جهالا ، فسئلوا فأفتوا بغير علم فضلوا وأضلوا ". ( رواه البخاري ومسلم ).فإذا كانت هذه هي حال النبي - صلى الله عليه وسلم - مع أصحابه رضي الله عنهم ، فكيف بحال الصحابة رضوان الله عليهم ؟!لوحة تحكم العضو الرسائل الخاصة الاشتراكات المتواجدون الآن البحث في المنتدى الصفحة الرئيسية للمنتدى إسلاميات هدى الإسلام مرئيات وسمعيات المعارف الإسلامية محمد رسول الله شخصيات إسلامية شبهات وردود لماذا أسلموا ؟ ؟ الدعاء والصدقة والذكر والاستغفار الفتن والملاحم منابر الدعوة أبواب الدعوة شبكات تواصل دعوية خطب إسلامية أبواب التوبة و قصص التائبين المنتديات المسيحية نصرانيات لاهوتيات مناظرات وحوارات الأخبار والمقالات الأخبار الإسلام والعالم الإسلام في أوروبا الإسلام في أسيا الإسلام في أفريقيا المقالات منوعات مكتبة طريق الخلاص المكتبة العامة العروض التقدمية العلوم الإسلامية كتب ومراجع مسيحيةعنوان الموضوع المشاركة الأخيرة تاريخ بداية الموضوع عدد الردود عدد المشاهدات كاتب ال

In [23]:
print("Prompt:\n",textwrap.fill(text_prep, 200))
print("Generated Text:\n",textwrap.fill(gen_text[len(text_prep):], 200))

Prompt:
 ما معنى القدر
Generated Text:
  ؟ قال : " إن الله لا يقبض العلم انتزاعا ينتزعه من العباد ، ولكن يقبض العلم بقبض العلماء ، حتى إذا لم يبق عالما اتخذ الناس رؤوسا جهالا ، فسئلوا فأفتوا بغير علم فضلوا وأضلوا ". ( رواه البخاري ومسلم
).فإذا كانت هذه هي حال النبي - صلى الله عليه وسلم - مع أصحابه رضي الله عنهم ، فكيف بحال الصحابة رضوان الله عليهم ؟!لوحة تحكم العضو الرسائل الخاصة الاشتراكات المتواجدون الآن البحث في المنتدى الصفحة
الرئيسية للمنتدى إسلاميات هدى الإسلام مرئيات وسمعيات المعارف الإسلامية محمد رسول الله شخصيات إسلامية شبهات وردود لماذا أسلموا ؟ ؟ الدعاء والصدقة والذكر والاستغفار الفتن والملاحم منابر الدعوة أبواب
الدعوة شبكات تواصل دعوية خطب إسلامية أبواب التوبة و قصص التائبين المنتديات المسيحية نصرانيات لاهوتيات مناظرات وحوارات الأخبار والمقالات الأخبار الإسلام والعالم الإسلام في أوروبا الإسلام في أسيا
الإسلام في أفريقيا المقالات منوعات مكتبة طريق الخلاص المكتبة العامة العروض التقدمية العلوم الإسلامية كتب ومراجع مسيحيةعنوان الموضوع المشاركة الأخيرة تاريخ بداية الموضوع عدد ال